In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
import pickle
import os
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential    # For sequential models
from tensorflow.keras.layers import Dense         # For nurons or nodes
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard  # TB for display info of some graphs
import datetime
from scikeras.wrappers import KerasClassifier
%matplotlib inline

In [12]:
dataset=pd.read_csv('../Data/Churn_Modelling.csv')
# drop irrelevent data
dataset = dataset.drop(columns=['RowNumber','CustomerId','Surname'], axis=1)


In [15]:
gender_encoder = LabelEncoder()
geo_oh_encoder = OneHotEncoder(sparse_output=False)
dataset['Gender'] = gender_encoder.fit_transform(dataset['Gender'])
encoded_data = geo_oh_encoder.fit_transform(dataset['Geography'].values.reshape(-1, 1))
geo_oh_encoder.get_feature_names_out(['Geography'])
geography_encoded_df = pd.DataFrame(encoded_data, columns=geo_oh_encoder.get_feature_names_out(['Geography']))
dataset = pd.concat([dataset.drop(['Geography'], axis=1), geography_encoded_df], axis=1)
dataset
dataset

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [16]:
### Dependent and Independent features selection
X = dataset.drop(columns=['Exited'], axis=1)
y = dataset['Exited']

### Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# Scales the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# create a model abd try different parameter

def create_model(neurons=32, layers=1):
    model=Sequential()
    # first hiden layer
    model.add(Dense(neurons, activation='relu', input_shape=(X_train[1],)))
    
    # create other layers:
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
        
    # output layer
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [21]:
## create a keras classifier
model = KerasClassifier(
    build_fn=create_model,
    epochs=50,
    batch_size=10,
    verbose=0
)

In [22]:
# here is grid search parameter
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2, 3],
    # 'batch_size': [10, 20],
    'epochs': [30, 50, 100]
}

In [ ]:
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    n_jobs=-1,
    cv=3
)

result = grid.fit(X_train, y_train)

# best results
print(f'Best {result.best_score_} : using {result.best_params_}')
print(f'Estimator: {result.best_estimator_}')